In [ ]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter


import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt
%matplotlib qt

import seaborn as sns 
import bk.load
import bk.compute
import bk.plot

import basefunction.anass_load as at
%run spatialInfo.ipynb

In [5]:
bk.load.current_session('Z:\Rat08\Rat08-20130717')
neurons,metadata = bk.load.loadSpikeData(bk.load.path)

Rat : 8 on day : 9
Working with session Rat08-20130717 @ Z:\Rat08\Rat08-20130717
Data already saved in Numpy format, loading them from here:
Rat08-20130717-neurons.npy
Rat08-20130717-metadata.npy


In [6]:
laps = {}
danger = scipy.io.loadmat('Rat08-20130717-LapType.mat')['aplaps'][0][0][0]
safe = scipy.io.loadmat('Rat08-20130717-LapType.mat')['safelaps'][0][0][0]

danger = nts.IntervalSet(danger[:,0],danger[:,1],time_units='s')
safe = nts.IntervalSet(safe[:,0],safe[:,1],time_units='s')

laps.update({'danger':danger,'safe':safe})

In [7]:
binned,times = bk.compute.binSpikes(neurons,1)
times = nts.Ts(times,time_units='s')

mean = {}
activity = {}
for lap in laps:
    b = laps[lap].in_interval(times)
    activity.update({lap:binned[:,b>=0]})
    mean.update({lap:np.mean(activity[lap],1)})

pval = scipy.stats.ttest_ind(activity['danger'],activity['safe'],1).pvalue

In [25]:
hpc_neurons = neurons[(metadata.Region == 'Hpc') & (metadata.Type == 'Pyr')]
bla_neurons = neurons[(metadata.Region == 'BLA') & (metadata.Type == 'Pyr')]

In [10]:
for i in range(pval.shape[0]):
    if (mean['danger'][i]>mean['safe'][i]) and pval[i]<0.05:
        plt.figure()
        plt.title('#'+str(i)+metadata.Region[i])
        plt.plot(activity['danger'][i])
        plt.plot(activity['safe'][i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
b,t = bk.compute.binSpikes(bla_neurons,0.25)

In [55]:
g = scipy.signal.gaussian(100,25)
g = g/g.sum()

In [56]:
mask = [2,3,7,9]
plt.figure()
# bk.plot.rasterPlot(bla_neurons[[5,14]],np.array([[9000,12000]]),'black',0.1,.2,.3)
for i in mask:
    plt.plot(t,np.convolve(b[i],g,'same'))
bk.plot.intervals(laps['danger'],col='r')
bk.plot.intervals(laps['safe'],col='g')

In [104]:
np.sum(mean['danger']>mean['safe'])

42

In [117]:
pval.pvalue

0.9240973692265089

In [43]:
laps['safe'].as_units('s')

,start,end
0,18.227200,37.862400
1,67.712000,70.528000
2,101.427200,104.345600
3,128.614399,131.430399
4,177.664000,180.352000
5,9410.355200,9416.473600
6,9556.172800,9561.497600
7,9814.016000,9823.897600
8,10420.684800,10436.480000
9,10538.214400,10560.358400


In [33]:
laps['safe'].tot_length(time_units='s')

154.7776

In [12]:
pos = bk.load.pos()

In [24]:
for n,i in enumerate(hpc_neurons):
    i = i.restrict(laps['safe'])
    computeRateMap(i,pos,50,20,plot = True)
    plt.suptitle('#'+str(n))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [30]:
for n,i in enumerate(bla_neurons):
    
    if n == 5:
        i = i.restrict(laps['danger'])
        computeRateMap(i,pos,50,20,plot = True)
        plt.suptitle('#'+str(n))
        plt.show()